In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(
     model_name="llama3-8b-8192" 
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage,AIMessage


## Message History

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
        
    return store[session_id]    
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [12]:
config={"configurable":{"session_id":"chat1"}}

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi ,My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [15]:
with_message_history.invoke(
    [HumanMessage(content="what my name?")],
    config=config
)

AIMessage(content='Your name is Krish, and you are a Chief AI Engineer!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 200, 'total_tokens': 214, 'completion_time': 0.011666667, 'prompt_time': 0.039415518, 'queue_time': 0.047311822, 'total_time': 0.051082185}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-02ebe0c4-6daa-4af6-882f-052bc2272663-0', usage_metadata={'input_tokens': 200, 'output_tokens': 14, 'total_tokens': 214})

In [25]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content=" my name  john")],
    config=config1
)
response.content


"You're sticking with it, John! Yeah, I'm convinced, your name is indeed John! I've got it memorized now!"

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="whats my name?")],
    config=config1
)
response.content

"John! I've got it memorized! Your name is John, and I'm not going to forget it!"

In [27]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

# chat Prompt

In [31]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpfu; assistant. Answer all question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model|parser

In [34]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is krish")]})

"Nice to meet you, Krish! It's great to have you here. Is there something I can help you with or would you like to chat about something in particular? I'm all ears and ready to assist you to the best of my abilities."

In [35]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [38]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="My name is krish.")],
    config=config
)
response

"Nice to meet you, Krish! It's great to know your name. From now on, I can address you by your name and personalize our conversation. How can I assist you today, Krish?"

In [39]:
response=with_message_history.invoke(
    [HumanMessage(content="what's my name?")],
    config=config
)
response

'I remember! Your name is Krish!'

In [40]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpfu; assistant. Answer all question to the nest of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model|parser

In [42]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Krish")],"language":"Hindi"})

'नमस्ते Krish! मैं आपकी मदद करने के लिए एक उपयोगी सहायक हूँ!如何可以मैं आपकी मदद करूँ?'

## Manage Conversion History

In [55]:
from langchain_core.messages import trim_messages,SystemMessage
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you're a good assostant"),
    HumanMessage(content="HI! I'm bob"),
    AIMessage(content="HI!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes"),
    
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assostant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="HI! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='HI!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [56]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [57]:
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
    
)

In [58]:
chain.invoke({
    "messages":messages+[HumanMessage(content="what ice cream do i like")],
    "language":"English"
})

AIMessage(content='you like vanilla ice cream!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 125, 'total_tokens': 132, 'completion_time': 0.005833333, 'prompt_time': 0.01548869, 'queue_time': 0.021552276000000002, 'total_time': 0.021322023}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-246c971b-fb0e-4ce3-8374-851e2084bb28-0', usage_metadata={'input_tokens': 125, 'output_tokens': 7, 'total_tokens': 132})